## Sentimaster

A sentiment analysis project for a competition-based hiring process. 

Here I investigate the application of a BERT-based approach for the tweets classification task. The choice was based on the reports of  	
<http://nlpprogress.com/english/sentiment_analysis.html> and specifically <https://doi.org/10.48550/arXiv.1905.05583>.

Nonetheless, a baseline approach using TF-IDF preprocessing with a random forest classifier was also implemented to compare the adequacy of the more sophisticated BERT-based strategy.



In [53]:
import pandas as pd

random_state = 42
train_file = "/home/colombelli/temp/applications/ey/data-set/train_complete.csv"
df = pd.read_csv(train_file)
df.head()

,tweet,label
0,"""QT @user In the original draft of the 7th boo...",2
1,"""Ben Smith / Smith (concussion) remains out of...",1
2,Sorry bout the stream last night I crashed out...,1
3,Chase Headley's RBI double in the 8th inning o...,1
4,@user Alciato: Bee will invest 150 million in ...,2


## Investigation of the basic dataset properties

In [11]:
print("Number of samples: ", len(df))
print("Labels:")
print(df['label'].value_counts())
print("\nTweet NaN values: ", df['tweet'].isna().sum())

Number of samples:  47615
Labels:
1    21542
2    18668
0     7405
Name: label, dtype: int64

Tweet NaN values:  0


## Data preprocessing 

I will use a baseline approach and compare to a state-of-the-art approach for sentiment analysis. 
The data preprocessing for the TF-IDF approach used with the baseline algorithm is heavier than the preprocessing performed in the data used by BERT.

In [41]:
import re
import string
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer 
from nltk.stem import PorterStemmer


tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                            reduce_len=True)
stemmer = PorterStemmer() 

# There are lots of these functions available on the internet
def text_preprocessing_tfidf(text):

    # Remove @mentions
    text = re.sub(r'(@.*?)[\s]', ' ', text)
    # Remove old style retweet text "RT"
    text = re.sub(r'^RT[\s]+', '', text)
    # Remove hyperlinks
    text = re.sub(r'https?://[^\s\n\r]+', '', text)
    # Remove the hash # sign from hashtags
    text = re.sub(r'#', '', text)

    text_clean = []
    for word in tokenizer.tokenize(text):
        if (word not in stopwords.words('english') and  # Remove stopwords
            word not in string.punctuation):  # Remove punctuation

            stem_word = stemmer.stem(word) # happy, happiness, etc -> happi            
            text_clean.append(stem_word)

    return " ".join(text_clean)


# This process can take some time and could be improved
def get_tfidf_preprocessed_dataset(df):
    df['tweet'] = df['tweet'].map(text_preprocessing_tfidf)
    return df

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/colombelli/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [43]:
tfidf_df = get_tfidf_preprocessed_dataset(df)
tfidf_df.head()

,tweet,label
0,qt origin draft 7th book remu lupin surviv bat...,2
1,ben smith smith concuss remain lineup thursday...,1
2,sorri bout stream last night crash tonight sur...,1
3,chase headley' rbi doubl 8th inning david pric...,1
4,alciato bee invest 150 million januari anoth 2...,2


In [45]:
def text_preprocessing_bert(text):

    # Remove @mentions
    text = re.sub(r'(@.*?)[\s]', ' ', text)
    # Replace '&amp;' with '&'
    text = re.sub(r'&amp;', '&', text)
    # Remove trailing whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    return text


def get_bert_preprocessed_dataset(df):
    df['tweet'] = df['tweet'].map(text_preprocessing_bert)
    return df

In [46]:
bert_df = get_bert_preprocessed_dataset(df)
bert_df.head()

,tweet,label
0,qt origin draft 7th book remu lupin surviv bat...,2
1,ben smith smith concuss remain lineup thursday...,1
2,sorri bout stream last night crash tonight sur...,1
3,chase headley' rbi doubl 8th inning david pric...,1
4,alciato bee invest 150 million januari anoth 2...,2


## Baseline evaluation

In [54]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

X = tfidf_df['tweet'].values
y = tfidf_df['label'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, 
                                                    random_state=random_state)

tf_idf = TfidfVectorizer(ngram_range=(1, 3),
                         binary=True)

X_train_tfidf = tf_idf.fit_transform(X_train)
X_test_tfidf = tf_idf.transform(X_test)

In [59]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

clf = RandomForestClassifier(max_depth=2, random_state=random_state)
clf.fit(X_train_tfidf, y_train)
y_pred = clf.predict(X_test_tfidf)

f1 = precision_recall_fscore_support(y_test, y_pred, average='macro', beta=1)[2]
print("Model performance (F1):", f1)
print("Accuracy:", accuracy_score(y_test, y_pred))  

Model performance (F1): 0.20529961730368648
Accuracy: 0.44498110037799243


/home/colombelli/miniconda3/envs/py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## BERT state-of-the-art evaluation